This notebook provides examples to go along with the [textbook](https://underactuated.csail.mit.edu/contact.html).  I recommend having both windows open, side-by-side!


In [ ]:
import matplotlib.pyplot as plt
import mpld3
import numpy as np
from IPython.display import display
from pydrake.all import DirectCollocation, MathematicalProgram, Solve
from pydrake.examples import RimlessWheel

from underactuated import running_as_notebook

if running_as_notebook:
    mpld3.enable_notebook()

# Trajectory optimization for the Rimless Wheel


In [ ]:
def rimless_wheel_limit_cycle():
    plant = RimlessWheel()
    context = plant.CreateDefaultContext()

    params = context.get_numeric_parameter(0)
    slope = params.slope()
    alpha = np.pi / params.number_of_spokes()

    dircol = DirectCollocation(
        plant,
        context,
        num_time_samples=15,
        minimum_time_step=0.01,
        maximum_time_step=0.1,
        assume_non_continuous_states_are_fixed=True,
    )
    prog = dircol.prog()

    dircol.AddEqualTimeIntervalsConstraints()

    dircol.AddConstraintToAllKnotPoints(dircol.state()[0] >= slope - alpha)
    dircol.AddConstraintToAllKnotPoints(dircol.state()[0] <= slope + alpha)

    prog.AddConstraint(dircol.initial_state()[0] == slope - alpha)
    prog.AddConstraint(dircol.final_state()[0] == slope + alpha)

    prog.AddConstraint(
        dircol.initial_state()[1] == dircol.final_state()[1] * np.cos(2 * alpha)
    )

    result = Solve(prog)
    assert result.is_success()

    x_trajectory = dircol.ReconstructStateTrajectory(result)

    x_knots = np.hstack(
        [
            x_trajectory.value(t)
            for t in np.linspace(
                x_trajectory.start_time(), x_trajectory.end_time(), 100
            )
        ]
    )

    fig, ax = plt.subplots()
    ax.plot(x_knots[0, :], x_knots[1, :])
    ax.plot([x_knots[0, 0], x_knots[0, -1]], [x_knots[1, 0], x_knots[1, -1]], "--")

    # Plot the energy contours.
    nq = 151
    nqd = 151
    mgl = params.mass() * params.gravity() * params.length()
    q = np.linspace(-0.5, 0.5, nq)
    qd = np.linspace(-0.5, 2, nqd)
    Q, QD = np.meshgrid(q, qd)
    Energy = 0.5 * params.mass() * params.length() ** 2 * QD**2 + mgl * np.cos(Q)
    ax.contour(
        Q,
        QD,
        Energy,
        alpha=0.5,
        linestyles="dashed",
        colors="black",
        linewidths=0.5,
    )

    ax.set_xlabel("theta")
    ax.set_ylabel("thetadot")
    ax.axis([-0.5, 0.5, 0, 2])
    ax.set_title("Limit Cycle of the Rimless Wheel (w/ contours of " "constant energy)")
    display(mpld3.display())


rimless_wheel_limit_cycle()